# VishwamAI QwQ-32B Distillation

Memory-efficient distillation from [Qwen/QwQ-32B](https://huggingface.co/Qwen/QwQ-32B) with chunked loading.

In [ ]:
import os
import jax
import jax.numpy as jnp
from omegaconf import OmegaConf
import aim
import gc
from huggingface_hub import snapshot_download

from vishwamai.model import VishwamAIModel, ModelConfig
from vishwamai.qwen_distiller import QwenDistillationTrainer
from vishwamai.qwen_data import QwenDataLoader

# Clear any existing cache
jax.clear_caches()
gc.collect()

# Print device info
print(f"JAX devices: {jax.devices()}")
print(f"Number of devices: {jax.device_count()}")

In [ ]:
# Memory-efficient settings
CHUNK_SIZE = 32  # Size of chunks for loading
BATCH_SIZE = 1   # Per device batch size
GRAD_ACCUM_STEPS = 16  # Gradient accumulation steps

# Load and update configuration
config = OmegaConf.load('configs/distillation_config.yaml')
config.training.batch_size = BATCH_SIZE
config.training.gradient_accumulation_steps = GRAD_ACCUM_STEPS

print(f"Configuration loaded with:")
print(f"- Chunk size: {CHUNK_SIZE}")
print(f"- Batch size per device: {BATCH_SIZE}")
print(f"- Gradient accumulation steps: {GRAD_ACCUM_STEPS}")

In [ ]:
# Download QwQ model with progress tracking
print("Downloading QwQ-32B model files...")
qwq_path = snapshot_download(
    "Qwen/QwQ-32B",
    allow_patterns=["*.safetensors", "config.json", "tokenizer.model"],
    local_files_only=False,
    resume_download=True
)

# Verify shard count
shard_files = [f for f in os.listdir(qwq_path) if f.endswith('.safetensors')]
print(f"Found {len(shard_files)} safetensor shards")
assert len(shard_files) == 14, f"Expected 14 safetensor files, found {len(shard_files)}"

In [ ]:
# Initialize data loader with memory-efficient settings
loader = QwenDataLoader(
    safetensor_dir=qwq_path,
    batch_size=BATCH_SIZE,
    max_sequence_length=config.distillation.teacher_model.config.max_position_embeddings,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    chunk_size=CHUNK_SIZE
)

print("Data loader initialized with chunked loading")

In [ ]:
# Initialize models
print("Initializing models...")
teacher_model = VishwamAIModel(ModelConfig(**config.distillation.teacher_model.config))
student_model = VishwamAIModel(ModelConfig(**config.distillation.student_model.config))

print("\nLoading QwQ model weights in chunks...")
params = loader.load_all_shards()  # This now uses chunked loading
teacher_model = teacher_model.bind({'params': params})

# Clear memory after loading
jax.clear_caches()
gc.collect()

print("Models initialized successfully")

In [ ]:
# Initialize trainer with experiment tracking
aim_run = aim.Run(
    experiment="VishwamAI-QwQ-Distillation",
    log_system_params=True
)
aim_run.set_params({
    "teacher_model": "QwQ-32B",
    "student_model": "VishwamAI-7B",
    "chunk_size": CHUNK_SIZE,
    "batch_size_per_device": BATCH_SIZE,
    "gradient_accumulation_steps": GRAD_ACCUM_STEPS,
    **OmegaConf.to_container(config, resolve=True)
})

trainer = QwenDistillationTrainer(
    teacher_model=teacher_model,
    student_model=student_model,
    cfg=config
)

rng = jax.random.PRNGKey(42)
state = trainer.create_train_state(rng)
print("Training setup complete")

In [ ]:
# Training loop with memory management
from tqdm.notebook import tqdm
import time

try:
    for epoch in range(5):  # 5 epochs
        print(f"\nEpoch {epoch + 1}/5")
        epoch_start = time.time()
        
        # Process shards sequentially with memory cleanup
        for shard_name, shard_params in tqdm(loader.get_shard_stream(), desc="Processing QwQ shards"):
            # Accumulate gradients over multiple steps
            accumulated_gradients = None
            
            for accum_step in range(GRAD_ACCUM_STEPS):
                # Create batch for current accumulation step
                batch = loader.create_training_batch(
                    input_ids=shard_params['input_ids'],
                    labels=shard_params.get('labels')
                )
                
                # Training step
                state, metrics, grads = trainer.train_step_with_grads(
                    state=state,
                    batch=batch,
                    rng=rng
                )
                
                # Accumulate gradients
                if accumulated_gradients is None:
                    accumulated_gradients = grads
                else:
                    accumulated_gradients = jax.tree_map(
                        lambda x, y: x + y,
                        accumulated_gradients,
                        grads
                    )
                
                # Clear intermediate memory
                if accum_step % 4 == 0:
                    jax.clear_caches()
                    gc.collect()
            
            # Apply accumulated gradients
            accumulated_gradients = jax.tree_map(
                lambda x: x / GRAD_ACCUM_STEPS,
                accumulated_gradients
            )
            state = state.apply_gradients(grads=accumulated_gradients)
            
            # Log metrics
            if state.step % config.training.logging_steps == 0:
                aim_run.track(
                    metrics,
                    step=state.step,
                    context={
                        'shard': shard_name,
                        'epoch': epoch,
                        'memory_usage': jax.numpy.zeros(1).device_buffer.physical_memory_allocated()
                    }
                )
                print(f"Step {state.step}: loss={metrics['loss']:.4f}")
            
            # Save checkpoint
            if state.step % config.training.save_steps == 0:
                ckpt_path = f"checkpoints/step_{state.step}"
                trainer.save_checkpoint(
                    state=state,
                    path=ckpt_path,
                    extra_info={
                        'epoch': epoch,
                        'shard': shard_name,
                        'metrics': metrics
                    }
                )
                aim_run.track_artifact(ckpt_path, name="checkpoints")
            
            # Memory cleanup after each shard
            del accumulated_gradients
            jax.clear_caches()
            gc.collect()
        
        epoch_time = time.time() - epoch_start
        print(f"Epoch completed in {epoch_time:.2f}s")

except KeyboardInterrupt:
    print("Training interrupted, saving checkpoint...")
    trainer.save_checkpoint(state, "checkpoints/interrupted")
finally:
    aim_run.close()

In [ ]:
# Save final model with memory usage stats
final_path = "final_vishwamai_model"

# Clear memory before saving
jax.clear_caches()
gc.collect()

trainer.save_model(
    state=state,
    path=final_path,
    config_override={
        "parent_model": "Qwen/QwQ-32B",
        "distillation_version": "v1.0",
        "training_config": {
            "chunk_size": CHUNK_SIZE,
            "batch_size_per_device": BATCH_SIZE,
            "gradient_accumulation_steps": GRAD_ACCUM_STEPS,
            "peak_memory_usage": float(jax.numpy.zeros(1).device_buffer.physical_memory_allocated())
        }
    }
)
print(f"Distilled model saved to {final_path}")